In [ ]:
import chromadb
from llama_index.core import PromptTemplate
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.chroma import ChromaVectorStore
from pathlib import Path
import os
import hashlib
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from slack_bolt.adapter.flask import SlackRequestHandler
from slack_bolt import App
from dotenv import find_dotenv, load_dotenv
from flask import Flask, request

In [4]:
#Check to see if Llama has been initialized
#llm = Ollama(model="llama3.1:latest")
llm = Ollama(model="llama3.1:latest", request_timeout=3000.0)
#response = llm.complete("What is linear regression?")
#print(response)

In [5]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
current_dir = os.getcwd()
data_folder= os.path.join(current_dir,"data")
chroma_client = chromadb.PersistentClient(path=data_folder)

In [18]:
# Delete the existing collection if it exists
if "ollama4" in chroma_client.list_collections():
    chroma_client.delete_collection("ollama4")

In [10]:
# Define the path to the folder containing the PDFs
pdf_folder_path = "C:\AI BootCamp\MyRepo\RAG_LLAM3\PDF"
txt_folder_path = "C:\AI BootCamp\MyRepo\RAG_LLAM3\TXT"

# Check if the directorys exist
if not os.path.isdir(pdf_folder_path):
    raise ValueError(f"Directory {pdf_folder_path} does not exist.")

if not os.path.isdir(txt_folder_path):
    raise ValueError(f"Directory {txt_folder_path} does not exist.")

# Gather all PDF and TXT files in the folder
pdf_files = list(Path(pdf_folder_path).glob('*.pdf'))
txt_files = list(Path(txt_folder_path).glob('*.txt'))

# Initialize a reader to read PDF files
pdf_reader = SimpleDirectoryReader(input_dir= pdf_folder_path, recursive=True)
txt_reader = SimpleDirectoryReader(input_dir= txt_folder_path, recursive=True)

# Custom class to mimic PDF structure for TXT files
class TxtDocument:
    def __init__(self, text, doc_id):
        self.text = text
        self.doc_id = doc_id
        self.id_ = doc_id  # Add id_ attribute

    def get_doc_id(self):
        return self.doc_id

    def hash(self):
        return hashlib.md5(self.text.encode('utf-8')).hexdigest()
    
    def get_metadata_str(self, mode=None):
        if mode:
            return f"Document ID: {self.doc_id}, Mode: {mode}"
        return f"Document ID: {self.doc_id}"
    
    def get_content(self, metadata_mode=None):
        if metadata_mode:
            return f"Content: {self.text}, Metadata Mode: {metadata_mode}"
        return self.text
    
    def as_related_node_info(self):
        return {
            "doc_id": self.doc_id,
            "text": self.text,
            "metadata": self.get_metadata_str()
        }
    
# Initialize an empty list to store PDF content
all_content = []

# Loop through each PDF and TXT file and extract content
for pdfs in pdf_reader.iter_data():
    for pdf in pdfs:
        pdf.text = pdf.text.upper()
        all_content.append(pdf)

# Loop through each TXT file and extract content
for txt_file in txt_files:
    with open(txt_file, 'r', encoding='utf-8') as file:
        txt_content = file.read()
        txt_doc = TxtDocument(text=txt_content.upper(), doc_id=str(txt_file))
        all_content.append(Document(text=txt_doc.get_content(), doc_id=txt_doc.get_doc_id()))


# Create a unified index
#chroma_client = chromadb.EphemeralClient()




In [ ]:

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [20]:

#Create pipeline for indexing and retrieving documents using vector database
chroma_collection = chroma_client.create_collection("ollama5")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    all_content, 
    storage_context=storage_context, 
    embed_model=embed_model, 
    transformations=[SentenceSplitter(chunk_size=256, chunk_overlap=10)],
    

)

In [54]:
# Load environment variables from .env file
load_dotenv(find_dotenv())

# Set Slack API credentials
SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]
SLACK_BOT_USER_ID = os.environ["SLACK_BOT_USER_ID"]

# Initialize the Slack app
app = App(token=SLACK_BOT_TOKEN)

# Initialize the Flask app
# Flask is a web application framework written in Python
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

In [55]:
def get_bot_user_id():
    """
    Get the bot user ID using the Slack API.
    Returns:
        str: The bot user ID.
    """
    try:
        # Initialize the Slack client with your bot token
        slack_client = WebClient(token=os.environ["SLACK_BOT_TOKEN"])
        response = slack_client.auth_test()
        return response["user_id"]
    except SlackApiError as e:
        print(f"Error: {e}")


def my_function(text):
    """
    Custom function to process the text and return a response.
    In this example, the function converts the input text to uppercase.

    Args:
        text (str): The input text to process.

    Returns:
        str: The processed text.
    """
    response = text.upper()
    return response

print(get_bot_user_id())

U07NH4J03D4


In [56]:
#Create Prompt Template
template = (
    "Imagine you are a professor teaching a class on Artifical Intelligence and "
    "you answer a student's questions about what you have gone over in previous lectures."
    "Here is some context from the Aritifical Intelligence class: \n"
    "related to the query::\n"
    "-----------------------------------------\n"
    "{context_str}\n"
    "-----------------------------------------\n"
    "Considering the above information, "
    "please respond to the following inquiry:\n\n"
    "Question: {query_str}\n\n"
    "Answer succinctly and ensure your response is "
    "clear to someone without a deep understaind of Artifical Intelligence."
    "The professor's name is Ryan."
)
qa_template = PromptTemplate(template)

In [31]:
#Create a text-based Question-Answering task the responds with the top 3 most similar documents
query_engine = index.as_query_engine(text_qa_template=qa_template, similarity_top_k=3)

In [32]:
print(query_engine.query("what is rohinis phone number"))

Student, I'm happy to help!

Your question is not related to Artificial Intelligence, but rather a straightforward inquiry about Rohini Patil's phone number.

To answer your question directly: According to the email you provided, Rohini Patil's phone number is +1 704 502 2766.


In [53]:
#Create a Slack WebClient for sending messages
@app.event("app_mention")
def handle_mentions(body, say):
    """
    Event listener for mentions in Slack.
    When the bot is mentioned, this function processes the text and sends a response.

    Args:@
        body (dict): The event data received from Slack.
        say (callable): A function for sending a response to the channel.
    """
    text = body["event"]["text"]

    mention = f"<@{SLACK_BOT_USER_ID}>"
    text = text.replace(mention, "").strip()

    say("Great question! Let me find the answer for you.")
    #response = query_engine.query(text)
    response = my_function(text)
    say(response.response)


@flask_app.route("/slack/events", methods=["POST"])
def slack_events():
    """
    Route for handling Slack events.
    This function passes the incoming HTTP request to the SlackRequestHandler for processing.

    Returns:
        Response: The result of handling the request.
    """
    return handler.handle(request)


# Run the Flask app
if __name__ == "__main__":
    flask_app.run(port=3000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit
127.0.0.1 - - [24/Sep/2024 14:38:02] "POST /slack/events HTTP/1.1" 401 -
127.0.0.1 - - [24/Sep/2024 14:38:21] "POST /slack/events HTTP/1.1" 401 -
127.0.0.1 - - [24/Sep/2024 14:41:09] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:41:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:41:33] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:41:51] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:41:56] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:53:46] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:53:49] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:53:50] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Sep/2024 14:55:22] "POST /slack/events HTTP/1.1" 401 -
127.0.0.1 - - [24/Sep/2024 14:55:35] "POST /slack/events HTTP/1.1" 401 -
127.0.0.1 - - [24/Sep/2024 14:55:40] "POST /slack/events HTTP/1.1" 401 -
